In [3]:
import cv2
import numpy as np
from keras.models import load_model

def webcam_prediction(model_path, consecutive_frames=10):
    # Load the trained model
    model = load_model(model_path)

    # Initialize the webcam
    cap = cv2.VideoCapture(0)

    # Set the dimensions and position of the rectangle
    rect_x, rect_y, rect_width, rect_height = 400, 0, 200, 200

    # Initialize variables for tracking predictions
    last_predictions = []
    current_prediction = None
    consecutive_count = 0

    while True:
        # Capture frame from webcam
        ret, frame = cap.read()

        # Draw the rectangle on the frame
        cv2.rectangle(frame, (rect_x, rect_y), (rect_x + rect_width, rect_y + rect_height), (0, 255, 0), 2)

        # Preprocess the frame for prediction
        region_of_interest = frame[rect_y:rect_y+rect_height, rect_x:rect_x+rect_width]
        input_data = preprocess_image(region_of_interest)

        # Make prediction
        prediction = model.predict(input_data)

        # Get the predicted number (assuming it's a classification task)
        predicted_number = np.argmax(prediction)

        # Track predictions over consecutive frames
        last_predictions.append(predicted_number)
        if len(last_predictions) > consecutive_frames:
            last_predictions.pop(0)

        # Check if a certain number of consecutive frames predict the same number
        if last_predictions.count(predicted_number) >= consecutive_frames / 2:
            current_prediction = predicted_number
            consecutive_count = consecutive_frames
        else:
            consecutive_count -= 1
            if consecutive_count <= 0:
                current_prediction = None

        # Display the frame with the predicted number
        if current_prediction is not None:
            cv2.putText(frame, f'Predicted Number: {current_prediction}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        cv2.imshow('Webcam Prediction', frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            #save the frame
            cv2.imwrite('frame.jpg', input_data)
            break

        # Delay to control the frame rate
        cv2.waitKey(1)

    # Release the webcam and close all windows
    cap.release()
    cv2.destroyAllWindows()

# Function to preprocess the image for prediction
def preprocess_image(frame):
    # Resize image to match the input size of the model
    resized_frame = cv2.resize(frame, (64, 64))
    # Mirror image
    #resized_frame = cv2.flip(resized_frame, 1)
    # Convert to grayscale
    gray_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)
    # Normalize pixel values to be between 0 and 1
    normalized_frame = gray_frame / 255.0
    # Expand dimensions to match the input shape of the model
    input_data = np.expand_dims(np.expand_dims(normalized_frame, axis=0), axis=-1)
    return input_data




In [6]:
webcam_prediction(r'C:\Users\nitro\OneDrive\Desktop\Final_Computer_Vision\Models\Model_1.h5')

1/1 [==============================] - 0s 29ms/step


In [1]:
import cv2
import numpy as np
from keras.models import load_model
import concurrent.futures

def webcam_multi_model_prediction(model_paths, consecutive_frames=5):
    # Load all the models
    models = [load_model(model_path) for model_path in model_paths]

    # Initialize the webcam
    cap = cv2.VideoCapture(0)

    # Set the dimensions and position of the rectangle
    rect_x, rect_y, rect_width, rect_height = 400, 0, 200, 200

    # Initialize variables for tracking predictions
    last_predictions = {model_path: [] for model_path in model_paths}
    current_prediction = None
    consecutive_count = consecutive_frames

    while True:
        # Capture frame from webcam
        ret, frame = cap.read()

        # Draw the rectangle on the frame
        cv2.rectangle(frame, (rect_x, rect_y), (rect_x + rect_width, rect_y + rect_height), (0, 255, 0), 2)

        # Use concurrent futures for parallel prediction
        with concurrent.futures.ThreadPoolExecutor() as executor:
            predictions = list(executor.map(lambda model: predict_single_frame(model, frame, rect_x, rect_y, rect_width, rect_height), models))

        # Use a voting mechanism to determine the final prediction
        final_prediction = voting_mechanism(predictions)

        # Track predictions over consecutive frames
        for i, model_path in enumerate(model_paths):
            last_predictions[model_path].append(predictions[i])
            if len(last_predictions[model_path]) > consecutive_frames:
                last_predictions[model_path].pop(0)

        # Check if a certain number of consecutive frames predict the same number
        if last_predictions[model_paths[0]].count(final_prediction) >= consecutive_frames / 2:
            current_prediction = final_prediction
            consecutive_count = consecutive_frames
        else:
            consecutive_count -= 1
            if consecutive_count <= 0:
                current_prediction = None

        # Display the frame with the predicted number
        if current_prediction is not None:
            cv2.putText(frame, f'Predicted Number: {current_prediction}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow('Webcam Multi-Model Prediction', frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        # Delay to control the frame rate
        cv2.waitKey(1)

    # Release the webcam and close all windows
    cap.release()
    cv2.destroyAllWindows()

def predict_single_frame(model, frame, rect_x, rect_y, rect_width, rect_height):
    # Preprocess the frame for prediction
    region_of_interest = frame[rect_y:rect_y+rect_height, rect_x:rect_x+rect_width]
    input_data = preprocess_image(region_of_interest)

    # Make prediction
    prediction = model.predict(input_data)

    # Get the predicted number (assuming it's a classification task)
    predicted_number = np.argmax(prediction)
    return predicted_number

def voting_mechanism(predictions):
    # Use a simple majority voting mechanism
    counts = np.bincount(predictions)
    final_prediction = np.argmax(counts)
    return final_prediction

# Function to preprocess the image for prediction
def preprocess_image(frame):
    # Resize image to match the input size of the model
    resized_frame = cv2.resize(frame, (64, 64))
    # Mirror image
    resized_frame = cv2.flip(resized_frame, 1)
    # Convert to grayscale
    gray_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)
    # Normalize pixel values to be between 0 and 1
    normalized_frame = gray_frame / 255.0
    # Expand dimensions to match the input shape of the model
    input_data = np.expand_dims(np.expand_dims(normalized_frame, axis=0), axis=-1)
    return input_data




In [2]:

model_paths = [
    r'C:\Users\nitro\OneDrive\Desktop\Final_Computer_Vision\Models\Model_1.h5',
    r'C:\Users\nitro\OneDrive\Desktop\Final_Computer_Vision\Models\Model_2.h5',
    r'C:\Users\nitro\OneDrive\Desktop\Final_Computer_Vision\Models\Model_3.h5'
]
webcam_multi_model_prediction(model_paths)



1/1 [==============================] - 0s 47ms/step


You should consider upgrading via the 'C:\Users\nitro\OneDrive\Desktop\Final_Computer_Vision\venv\Scripts\python.exe -m pip install --upgrade pip' command.
